In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import requests
from IPython.display import display , HTML

In [2]:
perfumes = pd.read_csv('perfumes_table.csv', encoding='latin-1') 

In [3]:
perfumes.duplicated(subset="title").sum()

np.int64(375)

In [4]:
perfumes.drop_duplicates(subset="title", inplace=True)
perfumes.drop_duplicates(subset="notes", inplace=True)

In [5]:
perfumes.insert(0, 'id', range(0, len(perfumes)))

In [6]:
perfumes.head(2)

,id,rating,notes,designer,reviews,description,url,title
0,0,5.00,"['Vanila', 'Madagascar Vanilla', 'Vanilla Abso...",fiorucci perfumes and colognes,[],Vanilla Scent by Fiorucci is a Amber Vanilla f...,https://www.fragrantica.com/perfume/Fiorucci/V...,Vanilla Scent Fiorucci for women and men
1,1,4.22,"['Mint', 'Lavender', 'Nutmeg', 'elemi', 'Cypre...",maiora parfum perfumes and colognes,[],Shady by Maiora Parfum is a fragrance for wome...,https://www.fragrantica.com/perfume/Maiora-Par...,Shady Maiora Parfum for women and men


In [7]:
perfumes=perfumes[['id','title','designer', 'notes','description','url']]

In [8]:
perfumes.tail(4)

,id,title,designer,notes,description,url
84140,79023,Ville Nuancielo for men,nuancielo perfumes and colognes,"['Spices', 'Green Notes', 'Bergamot', 'Petitgr...",Ville by Nuancielo is a Woody Floral Musk frag...,https://www.fragrantica.com/perfume/Nuancielo/...
84141,79024,Amo Ferragamo Flowerful Salvatore Ferragamo fo...,salvatore ferragamo perfumes and colognes,"['Campari', 'Yuzu', 'Black Currant', 'Pink Peo...",Amo Ferragamo Flowerful by Salvatore Ferragamo...,https://www.fragrantica.com/perfume/Salvatore-...
84142,79025,Black Jeans Versace for men,versace perfumes and colognes,"['Ginger', 'Tarragon', 'Lavender', 'Lemon', 'L...",Black Jeans by Versace is a Woody Floral Musk ...,https://www.fragrantica.com/perfume/Versace/Bl...
84143,79026,Snug Arcana Wildcraft for women and men,arcana wildcraft perfumes and colognes,"['Amber', 'Beeswax', 'Marshmallow', 'Candied F...",Snug by Arcana Wildcraft is a Amber fragrance ...,https://www.fragrantica.com/perfume/Arcana-Wil...


In [9]:
perfumes.isna().sum()

id             0
title          0
designer       0
notes          0
description    0
url            0
dtype: int64

In [10]:
import re
def convert_to_image_url(url):
    match = re.search(r'-(\d+)\.html$', url)
    if match:
        perfume_id = match.group(1)
        return f"https://fimgs.net/mdimg/perfume/375x500.{perfume_id}.jpg"
    return None  # or a placeholder image URL if needed

In [11]:
perfumes['img_url'] = perfumes['url'].apply(convert_to_image_url)

In [12]:
perfumes.shape

(79027, 7)

In [13]:
def extract_img_id(url):
    match = re.search(r'375x500\.(\d+)\.jpg$', str(url))
    if match:
        return int(match.group(1))
    return None

# Step 2: Create a new column with numeric ID
perfumes['img_id'] = perfumes['img_url'].apply(extract_img_id)

# Step 3: Sort using img_id numerically
sorted_perfumes = perfumes.sort_values(by='img_id', ascending=False)

# Step 4: View top 2
sorted_perfumes.head(2)

,id,title,designer,notes,description,url,img_url,img_id
14823,14245,French Fig The Saltworks Company for women,the saltworks company perfumes and colognes,"['Fig', 'Coconut Milk', 'Grass', 'Musk']",French Fig by The Saltworks Company is a fragr...,https://www.fragrantica.com/perfume/The-Saltwo...,https://fimgs.net/mdimg/perfume/375x500.87189.jpg,87189
37266,35420,Brown Sugar The Saltworks Company for women,the saltworks company perfumes and colognes,"['Brown sugar', 'Cherry Blossom', 'Peach', 'Am...",Brown Sugar by The Saltworks Company is a frag...,https://www.fragrantica.com/perfume/The-Saltwo...,https://fimgs.net/mdimg/perfume/375x500.87188.jpg,87188


In [32]:
perfumes=sorted_perfumes.iloc[1000:16000]

In [33]:
import ast
def collapse(L):
    L1 = []
    for i in ast.literal_eval(L):
        L1.append(i.replace(" ",""))
    return L1

In [34]:
def create_tags(perfumes):
    notes = perfumes['notes'].apply(collapse)
    description = perfumes['description'].apply(lambda x:x.split())
    designer = perfumes['designer'].apply(lambda x:x.split())
    perfumes['tags'] = notes + description + designer
    perfumes['tags'] = perfumes['tags'].apply(lambda x: " ".join(x)).str.lower()
    perfumes['notes'] = perfumes['notes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x.capitalize())
    perfumes['notes'] = perfumes['notes'].apply(
    lambda x: ', '.join(word.title() for word in x) if isinstance(x, list) else x
)


    perfumes.reset_index(drop=True, inplace=True)
    return perfumes

In [35]:
new_perfume = create_tags(perfumes)
new_perfume.head(2)

C:\Users\tesla\AppData\Local\Temp\ipykernel_10872\378312399.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfumes['tags'] = notes + description + designer
C:\Users\tesla\AppData\Local\Temp\ipykernel_10872\378312399.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfumes['tags'] = perfumes['tags'].apply(lambda x: " ".join(x)).str.lower()
C:\Users\tesla\AppData\Local\Temp\ipykernel_10872\378312399.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,id,title,designer,notes,description,url,img_url,img_id,tags
0,65655,CuÃ¬r de IrlandÃ©s The Dua Brand for women and...,the dua brand perfumes and colognes,"Juniper Berry, Pink Pepper, Tonka Bean, Clary ...",CuÃ¬r de IrlandÃ©s by The Dua Brand is a Leath...,https://www.fragrantica.com/perfume/The-Dua-Br...,https://fimgs.net/mdimg/perfume/375x500.85990.jpg,85990,juniperberry pinkpepper tonkabean clarysage bi...
1,27762,Crowned by Dua The Dua Brand for women,the dua brand perfumes and colognes,"Tonka Bean, Patchouli, Cedar",Crowned by Dua by The Dua Brand is a Amber Woo...,https://www.fragrantica.com/perfume/The-Dua-Br...,https://fimgs.net/mdimg/perfume/375x500.85989.jpg,85989,tonkabean patchouli cedar crowned by dua by th...


In [62]:
# #FIRST METHOD FOR TEXT PROCESSING
# from sklearn.feature_extraction.text import CountVectorizer
# #create a steamer from nltk to make words to their root form
# from nltk.stem import PorterStemmer
# ps = PorterStemmer()

# # Define custom tokenizer with stemming
# def tokenize_and_stem(text):
    
#     stems = [ps.stem(token) for token in text if token.isalpha()]  # stem only words
#     return stems
# # Create CountVectorizer with the custom tokenizer
# cv = CountVectorizer(max_features=5000, stop_words='english', tokenizer=tokenize_and_stem)


In [63]:
# vector = cv.fit_transform(new_perfume['tags']).toarray()

In [64]:
# from sklearn.metrics.pairwise import cosine_similarity
# similarity = cosine_similarity(vector)

In [65]:
# def recommend(perfume,top_n=5):
#     print(f"Recommendations for {perfume}:\n")
#     index = new_perfume[new_perfume['title'] == perfume].index[0]
#     distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     for i in distances[1:top_n+1]:
#         return new_perfume.iloc[i[0]]

In [66]:
# recommendations=recommend('Firewood Barba Brasil for men', top_n=5)

In [36]:
#SECOND METHOD FOR TEXT PROCESSING
vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = vectorizer.fit_transform(new_perfume['tags'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [37]:
def recommend_perfumes(selected_perfumes, n = 10):
    selected_indices = new_perfume[new_perfume['title'].isin(selected_perfumes)].index
    sim_scores = cosine_sim[selected_indices].mean(axis = 0)
    similar_indices = sim_scores.argsort()[::-1]
    recommended_indices = [i for i in similar_indices if i not in selected_indices][:n]

    return new_perfume.iloc[recommended_indices][['title', 'designer', 'description', 'notes', 'img_url']]

In [43]:
selected_perfumes = new_perfume.sample(n = 1, random_state = None)['title']
print('Selected Perfumes: ' + ', '.join(selected_perfumes) + '\n')

recommendations = recommend_perfumes(selected_perfumes)
recommendations

Selected Perfumes: Green Maremoto MATCA for women and men



,title,designer,description,notes,img_url
6848,Lurra MATCA for women and men,matca perfumes and colognes,Lurra by MATCA is a Citrus fragrance for women...,"Grapefruit, Neroli, Cinnamon, Vetiver, Cedar, ...",https://fimgs.net/mdimg/perfume/375x500.78663.jpg
1080,Green Clary Sage & Basil COLABO for women and men,colabo perfumes and colognes,Green Clary Sage & Basil by COLABO is a Aromat...,"Siam Benzoin, Cypress, Clary Sage, Basil",https://fimgs.net/mdimg/perfume/375x500.84851.jpg
6847,Smoked Sunday MATCA for women and men,matca perfumes and colognes,Smoked Sunday by MATCA is a Woody Aromatic fra...,"Green Mandarin, Rose, Ylang-Ylang, Sandalwood,...",https://fimgs.net/mdimg/perfume/375x500.78664.jpg
6850,Opera 9 MATCA for women and men,matca perfumes and colognes,Opera 9 by MATCA is a Amber Floral fragrance f...,"Bitter Orange, Blood Mandarin, Bergamot, Peach...",https://fimgs.net/mdimg/perfume/375x500.78661.jpg
6849,Oakmoss Tabac MATCA for women and men,matca perfumes and colognes,Oakmoss Tabac by MATCA is a Woody Aromatic fra...,"Bergamot, Cedar, Jasmine, Damask Rose, Opopona...",https://fimgs.net/mdimg/perfume/375x500.78662.jpg
6846,Woody Piperine MATCA for women and men,matca perfumes and colognes,Woody Piperine by MATCA is a Woody Spicy fragr...,"Bergamot, Bitter Orange, Black Pepper, Pink Pe...",https://fimgs.net/mdimg/perfume/375x500.78665.jpg
12960,Serene Green Tea ALT. Fragrances for women and...,alt. fragrances perfumes and colognes,Serene Green Tea by ALT. Fragrances is a Aroma...,"Green Tea, Bergamot, Jasmine, Orange Blossom, ...",https://fimgs.net/mdimg/perfume/375x500.72093.jpg
14723,Fresh Green Tea A'pieu for women,a'pieu perfumes and colognes,Fresh Green Tea by A'pieu is a Aromatic Green ...,"Bergamot, Mandarin Orange, Lemon, Orange, Gree...",https://fimgs.net/mdimg/perfume/375x500.70235.jpg
7727,Green Tea & Verbena Avon for women,avon perfumes and colognes,Green Tea & Verbena by Avon is a Aromatic Gree...,"Green Tea, Verbena",https://fimgs.net/mdimg/perfume/375x500.77723.jpg
6851,Pianista MATCA for women and men,matca perfumes and colognes,Pianista by MATCA is a Floral Fruity fragrance...,"Verbena, Bergamot, Ylang-Ylang, Jasmine, Rose,...",https://fimgs.net/mdimg/perfume/375x500.78660.jpg


In [44]:
def display_perfume_info(perfumes):
    html_content = ""
    
    for _, row in perfumes.iterrows():
        perfume_html = f"""
        <div style="display: flex; align-items: center; border: 1px solid #ddd; 
                    padding: 15px; margin: 10px 0; border-radius: 10px; 
                    background-color: #fafafa;">
            <img src="{row['img_url']}" width="150" height="150" 
                 style="border-radius: 10px; margin-right: 20px;">
                        <h3 style="margin: 0; color: #333;">{row['title'].capitalize()}</h3>
                <p style="margin: 5px 0; font-weight: bold; color: #555;">{row['designer'].capitalize()}</p>
                <p style="margin: 5px 0; color: #666;"><strong>Description:</strong> {row['description']}</p>
                <p style="margin: 5px 0; color: #666;"><strong>Ingredients:</strong> {row['notes']}</p></p>
            </div>
        </div>
        """
        html_content += perfume_html

    display(HTML(html_content))

In [45]:
display_perfume_info(recommendations)